<a href="https://colab.research.google.com/github/vasanramani/Great-Learning/blob/main/AppleHBR/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install -q langchain_community \
              langchain \
              langchain_openai \
              langchain-chroma \
              pymupdf==1.26.3 \
              tiktoken==0.9.0 \
              datasets==4.0.0 \
              evaluate==0.4.5

In [2]:
# Import core libraries
import os
import json
import requests
import chromadb

# Import libraries for working with PDFs and OpenAI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
from IPython.display import Markdown, display

# Import libraries for processing dataframes and text
import tiktoken
import pandas as pd

# Import LangChain components for data loading, chunking, embedding, and vector DBs
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

from datasets import Dataset
from langchain_openai import ChatOpenAI

modelToUse="gpt-4o-mini"
maxTokens=500
temp=0
topP=0.95

In [3]:
# Display Markdown formatted string in the notebook output.
# This function takes a string as input and uses IPython.display.Markdown
# to render it as Markdown, providing a more visually appealing output.
def printmd(string):
    display(Markdown(string))

In [4]:
# Identify the environment to switch the file paths either to use Google Drive or Local
import sys
# Check if the operating system is Windows
if sys.platform.startswith('win'):
    printmd("Running on Windows")
    resourcePath=""
else:
    # Assume it's a Linux-based environment, likely Google Colab
    printmd("Running on Linux most likely Colab")
    # Mount Google Drive for Colab environments to access files
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    # Define the base path for resources in Google Drive
    resourcePath="/content/drive/MyDrive/ColabAssignments/AppleHBR/"

Running on Windows

In [5]:
# Load Configuration for OPEN API

file_name = resourcePath + "config.json"
with open(file_name, 'r') as file:
    config = json.load(file)
    OPENAI_API_KEY = config.get("OPENAI_API_KEY")
    OPENAI_API_BASE = config.get("OPENAI_API_BASE")

# Store API credentials in environment variables for use by LangChain and OpenAI clients
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE

In [6]:
# Initialize OpenAI client
openAIClient = OpenAI()

In [7]:
def getDefaultResponse(user_prompt):
    # Access global variables for model configuration (model, max tokens, temperature, top_p)
    global modelToUse, maxTokens, temp, topP
    # Call the OpenAI API to get a chat completion
    completion = openAIClient.chat.completions.create(
        model=modelToUse, # Specify the AI model to use for the completion
        messages=[
            # Define the user's message to the AI model
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=maxTokens, # Set the maximum number of tokens in the response
        temperature=temp,     # Control the randomness of the output (lower = more deterministic)
        top_p=topP            # Control diversity via nucleus sampling (fraction of tokens to consider)
    )
    # Extract and return the content of the AI's response
    return completion.choices[0].message.content

In [8]:
def getGuidedResponse(system_prompt, user_prompt):
    # Access global variables for model configuration
    global modelToUse, maxTokens, temp, topP
    # Call the OpenAI API to get a chat completion with a system prompt for guidance
    completion = openAIClient.chat.completions.create(
        model=modelToUse, # Specify the AI model to use for the completion
        messages=[
            {"role": "system", "content": system_prompt}, # Define the system's role and guiding prompt
            {"role": "user", "content": user_prompt}     # Define the user's message to the AI model
        ],
        max_tokens=maxTokens, # Set the maximum number of tokens in the response
        temperature=temp,     # Control the randomness of the output
        top_p=topP            # Control diversity via nucleus sampling
    )
    # Extract and return the content of the AI's response
    return completion.choices[0].message.content

In [9]:
# Load the PDF from the local location and verify the content
pdfPath = resourcePath + "HBR_How_Apple_Is_Organized_For_Innovation.pdf"
pdfLoader = PyMuPDFLoader(pdfPath)
pdfContent = pdfLoader.load()

In [10]:
# Split the text as multiple chunks
textSplitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=2000, #Complete the code to define the chunk size
    chunk_overlap= 200 #Complete the code to define the chunk overlap
)
pdfChunks = pdfLoader.load_and_split(textSplitter)
pdfChunksLength = len(pdfChunks)
printmd(f"The PDF File is split into {pdfChunksLength} Chunks")

The PDF File is split into 11 Chunks

In [11]:
# Initialize the Embeddings and verify if it loaded properly and it is of the same size
embeddingModel = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE
)
embedding0 = embeddingModel.embed_query(pdfChunks[0].page_content)
embedding1 = embeddingModel.embed_query(pdfChunks[1].page_content)

isEmbeddingSameSize = len(embedding0) == len(embedding1)
printmd(f"Dimension of the embedding vector is {len(embedding0)}")
printmd(f"Both Embeddings are {"SAME" if isEmbeddingSameSize else "DIFFERNT"} size")

Dimension of the embedding vector is 1536

Both Embeddings are SAME size

In [12]:
# Create the vector database from the chunks
vectorDBDir="/content/vectordb"
if not os.path.exists(vectorDBDir):
    os.makedirs(vectorDBDir)
    printmd("Vector DB Directory is created")
vectorDB = Chroma.from_documents(
    documents=pdfChunks,
    embedding=embeddingModel,
    persist_directory=vectorDBDir
)
printmd("Vector DB contents is created")

Vector DB contents is created

In [13]:
# Load the vector DB
vectorDBStore = Chroma(
    embedding_function=embeddingModel,
    persist_directory=vectorDBDir
)
vectorDBStore.embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000025E58608590>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000025E58608EC0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base='https://aibe.mygreatlearning.com/openai/v1', openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [14]:
vectorDBStore.similarity_search("Who are the authors of this article and who published this article?", k=1)

[Document(id='4bfa1387-ff3d-408e-82a8-7fd941360c15', metadata={'file_path': 'HBR_How_Apple_Is_Organized_For_Innovation.pdf', 'keywords': '', 'author': '', 'producer': 'Adobe PDF Library 15.0 (via http://bfo.com/products/pdf?version=2.23.5-r33279)', 'creationDate': "D:20201005141842-04'00'", 'trapped': '', 'source': 'HBR_How_Apple_Is_Organized_For_Innovation.pdf', 'creationdate': '2020-10-05T14:18:42-04:00', 'page': 1, 'subject': '', 'title': '', 'format': 'PDF 1.6', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'encryption': 'Standard V2 R3 128-bit RC4', 'moddate': '2020-12-01T18:37:49+00:00', 'total_pages': 11, 'modDate': 'D:20201201183749Z'}, page_content='2\nHarvard Business Review\nNovember–December 2020\nThis article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.')]

In [15]:
# Initialize a retriever from the Chroma vector store
# This retriever will be used to fetch relevant document chunks based on a query.
vectorDBRetriever = vectorDBStore.as_retriever(
    search_type="similarity", # Specify that the search type should be 'similarity' to find semantically similar documents
    search_kwargs={"k": 1}    # Configure the search to return the top 1 most similar document chunk
)

In [16]:
def getRAGResponse(system_prompt, user_input,k=3):
    global qaUserMsgTemplate, modelToUse, maxTokens, temp, topP
    # Retrieve relevant document chunks using the vector database retriever
    matchingChunks = vectorDBRetriever.invoke(user_input)
    # Extract the page content from the retrieved document chunks
    contexts = [d.page_content for d in matchingChunks]
    # Combine the extracted contexts into a single complete context string
    completeContext = ". ".join(contexts)
    # Populate the user message template with the retrieved context and user's question
    userMessage = qaUserMsgTemplate.replace('{context}', completeContext)
    userMessage = userMessage.replace('{question}', user_input)
    # Generate the response using the OpenAI client
    try:
        response = openAIClient.chat.completions.create(
        model=modelToUse,   # Specify the model to be used for completion
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": userMessage}
        ],
        max_tokens=maxTokens,
        temperature=temp,
        top_p=topP
        )
        # Extract and print the generated text from the response
        response = response.choices[0].message.content.strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

In [17]:
def getResponse(system_prompt, user_question):
    global qaUserMsgTemplate # Ensure global access if not defined in function scope

    printmd(f"## Question: {user_question}")
    printmd(f"### System Prompt: {system_prompt}")

    printmd("### Default Response:")
    default_response = getDefaultResponse(user_question)
    if "JSON" in system_prompt or "XML" in system_prompt:
        print(default_response)
    else:
        printmd(default_response)

    printmd("### Guided Response (Default Prompt):")
    guided_response_default = getGuidedResponse(system_prompt, user_question)
    if "JSON" in system_prompt or "XML" in system_prompt:
        print(guided_response_default)
    else:
        printmd(guided_response_default)

    printmd("### RAG Response (Default Prompt):")
    rag_response_default = getRAGResponse(system_prompt, user_question)
    if "JSON" in system_prompt or "XML" in system_prompt:
        print(rag_response_default)
    else:
        printmd(rag_response_default)

    printmd("\n---\n") # Separator for clarity

In [18]:
question0 = "Who are the authors of this article and who published this article?"
question1 = "List down the three leadership characteristics in bulleted points and explain each one of the characteristics under two lines."
question2 = "Can you explain specific examples from the article where Apple's approach to leadership has led to successful innovations?"
sysPromptDefault="Your response must be friendly and informative."
sysPromptJSON="Your response must be friendly and informative. Create response in a JSON Format."
sysPromptXML="Your response must be friendly and informative. Create response in a XML Format."
qaUserMsgTemplate="### {context} $$$$"

# Below is the call to the getResponse method with the three different questions and different system prompts.
# The system prompts contains prompts to format the contents as default, JSON and XML   
# The output is documented as a well formatted markdown document.

getResponse(sysPromptDefault, question0)
getResponse(sysPromptDefault, question1)
getResponse(sysPromptDefault, question2)

getResponse(sysPromptJSON, question0)
getResponse(sysPromptJSON, question1)
getResponse(sysPromptJSON, question2)

getResponse(sysPromptXML, question0)
getResponse(sysPromptXML, question1)
getResponse(sysPromptXML, question2)

## Question: Who are the authors of this article and who published this article?

### System Prompt: Your response must be friendly and informative.

### Default Response:

I'm sorry, but I can't access external content or databases to retrieve specific articles or their authors. If you provide me with the title or some details about the article, I might be able to help you with general information or context related to the topic.

### Guided Response (Default Prompt):

...

### Guided Response (Default Prompt):

```xml
<response>
    <greeting>Hello!</greeting>
    <information>
        <introduction>
            Apple's approach to leadership has been pivotal in driving successful innovations throughout its history. Here are a few specific examples that highlight this.
        </introduction>
        <examples>
            <example>
                <title>iPhone Launch</title>
                <description>
                    Under Steve Jobs' leadership, the launch of the iPhone in 2007 revolutionized the smartphone industry. Jobs emphasized a focus on design and user experience, which led to a product that combined a phone, an iPod, and an internet communicator. This innovative approach not only transformed Apple's product line but also set new standards for mobile devices.
                </description>
            </example>
            <example>
                <title>Apple Retail Stores</title>
                <description>
                    The introduction of Apple Retail Stores in 

### RAG Response (Default Prompt):

```xml
<response>
    <greeting>Hello!</greeting>
    <information>
        <summary>
            This excerpt highlights Apple's unique organizational culture, emphasizing the importance of specialization and expertise among its leaders. Steve Jobs believed that managers should be experts in their fields, fostering an environment where world-class talent can thrive and learn from one another.
        </summary>
        <keyPoints>
            <point>
                <title>Expertise in Management</title>
                <description>Steve Jobs advocated for hiring managers who are experts in their areas rather than just professional managers, as this fosters a learning environment for employees.</description>
            </point>
            <point>
                <title>Example of Leadership</title>
                <description>Roger Rosner, VP of applications, exemplifies this principle by leading a team of experts in software applications, showcasing the value of deep expertise in


---


In [19]:
getResponse(sysPromptDefault, "What is an example of the attention to detail?")

## Question: What is an example of the attention to detail?

### System Prompt: Your response must be friendly and informative.

### Default Response:

An example of attention to detail can be seen in the work of a skilled chef preparing a gourmet dish. For instance, when plating a meal, the chef carefully considers the arrangement of each component on the plate, ensuring that colors contrast beautifully, textures are varied, and portions are balanced. They might also pay close attention to the garnishes, using microgreens or edible flowers to enhance the visual appeal. Additionally, the chef ensures that the flavors are harmonized, with each ingredient complementing the others, and that the cooking techniques are executed perfectly to achieve the desired doneness. This meticulous approach not only elevates the dining experience but also reflects the chef's commitment to quality and excellence.

### Guided Response (Default Prompt):

Attention to detail refers to the ability to notice and consider all the small elements that contribute to a larger task or project. Here’s an example:

Imagine a graphic designer working on a marketing campaign. They not only focus on the overall layout and color scheme but also pay close attention to the following details:

1. **Font Consistency**: Ensuring that the same font is used throughout the materials for a cohesive look.
2. **Alignment**: Making sure that all text and images are perfectly aligned, creating a clean and professional appearance.
3. **Color Matching**: Using the exact brand colors to maintain brand identity and recognition.
4. **Image Quality**: Selecting high-resolution images to avoid pixelation and ensure clarity.
5. **Proofreading**: Carefully checking for spelling and grammatical errors in all text before finalizing the design.

By focusing on these details, the designer enhances the overall quality of the campaign, making it more effective and visually appealing. This kind of meticulousness can significantly impact the success of a project!

### RAG Response (Default Prompt):

This excerpt provides a fascinating look into the collaborative and detail-oriented culture at Apple, particularly in the development of the iPhone's portrait mode feature. The challenges faced by the camera team highlight the importance of addressing even the rarest of corner cases to maintain the company's high standards of quality, which they refer to as "zero artifacts." 

The discussions between engineering, design, and marketing teams illustrate how diverse perspectives can lead to innovative solutions. For instance, the push for a live preview of the portrait effect before taking a photo was a significant enhancement that improved user experience. This kind of collaboration, where teams work together towards a common goal while respecting each other's expertise, is a hallmark of Apple's approach to product development.

Moreover, the concept of "accountability without control" is intriguing. It emphasizes the need for leaders to inspire and motivate cross-functional teams, even when they don't have direct authority over them. This can lead to what the article describes as "good mess," where creativity and collaboration flourish, resulting in successful outcomes.

Lastly, the attention to detail, such as the use of a "squircle" for rounded corners, showcases Apple's commitment to not just functionality but also aesthetics, which is crucial in their product design philosophy. Overall, this example underscores how a culture of collaboration, high standards, and attention to detail can drive innovation and success in a competitive landscape.


---


In [ ]:
# Generate actionable insights and business recommendations using the RAG pipeline
insights_prompt = (
    "Based on the article 'How Apple Is Organized for Innovation', generate 5 concise, actionable insights "
    "that product and engineering leaders can apply. For each insight provide a one-line action and a one-sentence rationale."
)

recommendations_prompt = (
    "Based on the same article, produce 7 concrete business recommendations for senior leadership "
    "and org design to improve innovation outcomes. For each recommendation provide a short rationale and a suggested first step."
)

# Get RAG-backed responses
actionable_insights = getRAGResponse(sysPromptDefault, insights_prompt)
business_recommendations = getRAGResponse(sysPromptDefault, recommendations_prompt)

# Display in notebook
printmd("## Actionable Insights")
printmd(actionable_insights)

printmd("## Business Recommendations")
printmd(business_recommendations)

# Persist results to a JSON file for reuse
output = {
    "actionable_insights": actionable_insights,
    "business_recommendations": business_recommendations
}

## Actionable Insights

It looks like you're referencing an article from the Harvard Business Review titled "How Apple Is Organized for Innovation" by Joel M. Podolny and Morten T. Hansen. This piece explores how Apple's organizational structure fosters a culture of innovation, emphasizing the importance of having experts lead teams of experts. 

The article likely discusses key elements such as collaboration, cross-functional teams, and the role of leadership in nurturing creativity and innovation within the company. Apple's approach to maintaining a dynamic and innovative environment is often cited as a significant factor in its success.

If you're interested in specific insights or themes from the article, feel free to ask! I'm here to help.

## Business Recommendations

The article "How Apple Is Organized for Innovation" by Joel M. Podolny and Morten T. Hansen, published in the Harvard Business Review, delves into the unique organizational structure of Apple Inc. It highlights how the company fosters a culture of innovation by empowering experts to lead their respective fields. 

Apple's approach emphasizes collaboration among specialists, allowing for a dynamic exchange of ideas and expertise. This structure not only enhances creativity but also ensures that decisions are made by those with the most relevant knowledge, leading to more effective problem-solving and product development.

The article provides insights into how Apple's organizational culture supports its mission to innovate continuously, making it a fascinating read for anyone interested in business strategy and organizational behavior. If you have any specific questions or need further information about the article or its themes, feel free to ask!